### Association rules by Top 5 popular Departments 
#### Department - Snacks

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [6]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [ ]:
combine_train_prior

In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="snacks"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [10]:
grouped_data

,order_id,product_name
18,1145,Goldfish Parmesan Baked Snack Crackers
19,1145,Little Bites Blueberry Muffin Pouches
20,1145,Milano Milk Chocolate Cookies
21,1145,Harvest Cheddar Multigrain Chips
26,1591,Organic Bunny Fruit Snacks Berry Patch
27,1591,Honey Graham Snacks
28,1591,Goldfish Pretzel Baked Snack Crackers
29,1591,Original Whole Grain Chips
30,1591,Chewy 25% Low Sugar Chocolate Chip Granola
31,1591,Nutty Bars


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
#relace NAN with 0
table1=table1.fillna(0)

In [13]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [14]:
basket_sets = table1.applymap(encode_units)

In [ ]:
basket_sets

In [15]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,0.023780,[100 Calorie Per Bag Popcorn]
1,0.023224,[Aged White Cheddar Baked Rice & Corn Puffs Gl...
2,0.013238,[Almond Nut & Rice Cracker Snacks]
3,0.021901,[Almonds & Sea Salt in Dark Chocolate]
4,0.014911,[Apple Cinnamon GoGo Squeez]
5,0.023311,[Apple Pie Fruit & Nut Food Bar]
6,0.018355,[Backyard Barbeque Potato Chips]
7,0.024248,[Baked Aged White Cheddar Rice and Corn Puffs]
8,0.010850,[Barbecue Potato Chips]
9,0.018406,[Blueberry Muffin Bar]


In [21]:
# # Getting the rules of association based on the minimum support selected
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)

In [22]:
# rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [90]:
# ## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
# final_result=rules[ (rules['lift'] > 1) &
#        (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
# final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [23]:
frequent_itemsets.to_csv("Department_SnacksApriori.csv",sep=',')

In [24]:
test1=pd.read_csv("Department_SnacksApriori.csv")

In [25]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [26]:
test1=test1.replace({"u'": ''}, regex=True)

In [ ]:
test1

In [28]:
test1['itemsets'] = test1['itemsets'].str.strip('()').str.strip('[]')
test1['itemsets'] = test1['itemsets'].str.strip('[]')
test1['itemsets'] = test1['itemsets'].str.strip("'")
test1['itemsets'] = test1['itemsets'].str.replace("'",'')

In [48]:
# test1['consequents'] = test1['consequents'].str.strip('()')
# test1['consequents'] = test1['consequents'].str.strip('[]')
# test1['consequents'] = test1['consequents'].str.strip("'")
# test1['consequents'] = test1['consequents'].str.replace("'",'')

In [30]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [31]:
test1

,support,itemsets
0,0.023780,100 Calorie Per Bag Popcorn
1,0.023224,Aged White Cheddar Baked Rice & Corn Puffs Glu...
2,0.013238,Almond Nut & Rice Cracker Snacks
3,0.021901,Almonds & Sea Salt in Dark Chocolate
4,0.014911,Apple Cinnamon GoGo Squeez
5,0.023311,Apple Pie Fruit & Nut Food Bar
6,0.018355,Backyard Barbeque Potato Chips
7,0.024248,Baked Aged White Cheddar Rice and Corn Puffs
8,0.010850,Barbecue Potato Chips
9,0.018406,Blueberry Muffin Bar


In [32]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_SnacksApriori.csv")

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Icelandic Style Skyr Blueberry Non-fat Yogurt,Vanilla Skyr Nonfat Yogurt,0.029076,0.025750,0.012142,0.417576,16.216821,0.011393,1.672752
1,Vanilla Skyr Nonfat Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,0.025750,0.029076,0.012142,0.471526,16.216821,0.011393,1.837222
2,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% All Natural Greek Strained Yogurt wit...,0.043433,0.031430,0.010926,0.251564,8.003830,0.009561,1.294125
3,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% with Strawberry Lowfat Greek Strained...,0.031430,0.043433,0.010926,0.347631,8.003830,0.009561,1.466297
4,Non Fat Raspberry Yogurt,Vanilla Skyr Nonfat Yogurt,0.024847,0.025750,0.009568,0.385058,14.953976,0.008928,1.584298
5,Vanilla Skyr Nonfat Yogurt,Non Fat Raspberry Yogurt,0.025750,0.024847,0.009568,0.371567,14.953976,0.008928,1.551720
6,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% with Strawberry Lowfat Greek Strained...,0.032745,0.043433,0.017159,0.524006,12.064665,0.015737,2.009618
7,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% Lowfat Greek Strained Yogurt With Blu...,0.043433,0.032745,0.017159,0.395062,12.064665,0.015737,1.598933
8,Total 2% Lowfat Greek Strained Yogurt with Peach,Total 2% with Strawberry Lowfat Greek Strained...,0.031283,0.043433,0.014900,0.476298,10.966245,0.013541,1.826548
9,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% Lowfat Greek Strained Yogurt with Peach,0.043433,0.031283,0.014900,0.343059,10.966245,0.013541,1.474587


In [196]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()